**Thanks to @columbia2131 for sharing pickle file for faster loading of data. In this notebook, I will be using GroupKfold as well Group + Stratified Kfold to split the training data. I am open to feedbacks**

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
import seaborn as sns
from sklearn.model_selection import StratifiedKFold,GroupKFold
from collections import Counter, defaultdict
import random

from collections import Counter, defaultdict


In [ ]:
df = pd.read_pickle('../input/ump-train-picklefile/train.pkl')

In [ ]:
sns.displot(df['target'])

In [ ]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

*Creating Bins for the regression target variable.*

In [ ]:
num_bins = 12
df["bins"] = pd.cut(df["target"], bins=num_bins, labels=False)


In [ ]:
df['bins'].value_counts()

In [ ]:
X = df.drop(['target'],axis=1)

y = df['bins'].values
groups = np.array(df.time_id.values)  ## Grouping column


**Five fold Group + stratified split**

In [ ]:
df['fold'] = 0
for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(X, y, groups, k=5)):
    df.loc[val_ind,'fold'] = fold_ind
    print(f'validation fold {fold_ind} split finished')
df.drop(['bins'],axis=1).to_parquet('df_gs_5folds.parquet',index=False)

**Ten fold Group + stratified split**

In [ ]:
df['fold'] = 0
for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(X, y, groups, k=10)):
    df.loc[val_ind,'fold'] = fold_ind
    print(f'validation fold {fold_ind} split finished')
df.drop(['bins'],axis=1).to_parquet('df_gs_10folds.parquet',index=False)

In [ ]:
df.fold.value_counts()

**5 fold split with GroupKfold**

In [ ]:
df['fold'] = 0
gk = GroupKFold(n_splits=5)
for fold_ind, (dev_ind, val_ind) in enumerate(gk.split(X, y, groups)):
    df.loc[val_ind,'fold'] = fold_ind
    print(f'validation fold {fold_ind} split finished')
df.drop(['bins'],axis=1).to_parquet('df_g_5folds.parquet',index=False)

**10 folds split with GroupKfold**

In [ ]:
df['fold'] = 0
gk = GroupKFold(n_splits=10)
for fold_ind, (dev_ind, val_ind) in enumerate(gk.split(X, y, groups)):
    df.loc[val_ind,'fold'] = fold_ind
    print(f'validation fold {fold_ind} split finished')
df.drop(['bins'],axis=1).to_parquet('df_g_10folds.parquet',index=False)

In [ ]:
df.fold.value_counts()

In [ ]:
df.head()

**I have saved all folds in parquet files, Use pd.read_parquet() for much faster loading.**